# DEEPFACE EMBEDDING AND SAVE

## Move correct folder

In [22]:
%cd data_generation

[Errno 2] No such file or directory: 'data_generation'
/mnt/d/deepface-video-recognition/data_generation


## IMPORT LIB

In [23]:
from deepface import DeepFace

import concurrent.futures
import os
import pickle
import numpy as np
import faiss
import queue
import threading
import cv2
import matplotlib.pyplot as plt

## Initial Parameters and Function

In [24]:
fixed_dim = 2622
workers = 3
movies = ['Calloused_Hands', 'Liberty_Kid', 'like_me', 'losing_ground', 'Memphis']
movie = "like_me"
detect_faces_folder = "det_faces"
character_query_folder = "character_query"
character_emb_query_folder = "character_emb_query"
faces_folder = "faces"
faces_emb_folder = "faces_emb"
emb_folder_mapping = {
    "ArcFace": "ArcFace",
    "VGG-Face": "VggFace",
    "Facenet": "FaceNet",
    "GhostFaceNet": "GhostFaceNet"
}

embs = ["ArcFace", "VGG-Face", "Facenet", "GhostFaceNet"]

## Embedding Function

In [25]:
def create_file_path(source_file_path, source_folder, destination_folder):
    # Construct destination file path
    destination_file_path = os.path.join(destination_folder, os.path.relpath(source_file_path, source_folder))

    destination_dir = os.path.dirname(destination_file_path)
    # Create destination folder if not exists
    os.makedirs(destination_dir, exist_ok=True)
    return destination_file_path

In [26]:
def pad_embedding(embedding, target_dim):
    """Pad or truncate embedding to the target dimension."""
    if len(embedding) >= target_dim:
        return embedding[:target_dim]
    else:
        padding = np.zeros(target_dim - len(embedding))
        return np.concatenate((embedding, padding))

In [27]:
def attach_model_to_path(file_name, model):
    if "det_opencv" in file_name:
        new_filename = file_name.replace("det_opencv", f"det_opencv-emb_{model}")
    elif "det_fastmtcnn" in file_name:
        new_filename = file_name.replace("det_fastmtcnn", f"det_fastmtcnn-emb_{model}")
    elif "det_retinaface" in file_name:
        new_filename = file_name.replace("det_retinaface", f"det_retinaface-emb_{model}")
    else:
        new_filename = file_name
    return new_filename

In [28]:
def extract_emb_parallel(source_file_path, model_emb_file_path, model):
    global fixed_dim
    embeddings = DeepFace.represent(img_path = source_file_path,
                                        model_name = model,
                                       enforce_detection = False)
    original_embedding = np.array(embeddings[0]['embedding'], dtype=np.float32)
    padded_embedding = pad_embedding(original_embedding, fixed_dim)
    padded_embedding = np.expand_dims(padded_embedding, axis=0)  # Add batch dimension
    with open(model_emb_file_path, "wb") as f:
        pickle.dump(padded_embedding, f)
        print(f"Created embeddings: {source_file_path} -> {model_emb_file_path}")

In [29]:
def extract_emb(source_folder: str, des_em_folder: str, folder: str, model: str):
    # Traverse to all files and subfolder in source folder
    for root, _, files in os.walk(source_folder):
        with ThreadPoolExecutor(max_workers=workers) as executor:
            for file in files:
                # Construct source file path
                source_file_path = os.path.join(root, file)
                emb_file_path = create_file_path(source_file_path, source_folder, des_em_folder)
                model_emb_file_path = attach_model_to_path(os.path.splitext(emb_file_path)[0] +  ".pkl", model)
                if source_file_path.lower().endswith('.jpg') and os.path.exists(model_emb_file_path) == False:
                    executor.submit(extract_emb_parallel, source_file_path, model_emb_file_path, model)
                else:
                    print(f"{source_file_path} not end with jpg or {model_emb_file_path} exists")

## Create all faces img

In [30]:
def extract_faces_parallel(face_index_img_file_path, model_emb_file_path, extraction):
    global fixed_dim, cv2, emb_folder_mapping
    if os.path.exists(face_index_img_file_path) == False:
        cv2.imwrite(face_index_img_file_path, extraction)
        print(f"Created face detection: {face_index_img_file_path}")
    else:
        print(f"{face_index_img_file_path} exists")

In [31]:
def extract_faces_emb(source_folder: str, des_img_faces: str):
    global faces_folder, detect_faces_folder
    # Traverse to all files and subfolder in source folder
    for root, _, files in os.walk(source_folder):
        with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
            for file in files:
                # Construct source file path
                source_file_path = os.path.join(root, file)
                face_img_file_path = create_file_path(source_file_path, source_folder, des_img_faces).replace(".pkl", ".jpg")
                if source_file_path.lower().endswith('.pkl'):
                    with open(source_file_path, "rb") as f:
                        face_index = 1
                        extractions = pickle.load(f)
                        for extraction in extractions:
                            face_index_img_file_path = face_img_file_path.replace(".jpg", f"-face_{face_index}.jpg")
                            executor.submit(extract_faces_parallel, face_index_img_file_path, source_file_path, extraction)
                            # extract_emb_and_faces_parallel(face_index_img_file_path, des_img_faces, des_em_folder, source_file_path, model, extraction)
                            face_index += 1

In [32]:
def create_faces_emb():
    global movie, detect_faces_folder, faces_folder
    source_det_faces = f"../{detect_faces_folder}/{movie}"
    # des_emb_faces = f"../{faces_emb_folder}/{movie}"
    des_img_faces = f"../{faces_folder}/{movie}"
    extract_faces_emb(source_det_faces, des_img_faces)

In [ ]:
create_faces_emb()

In [ ]:
print("Done")

In [ ]:
# global movie
# source_det_faces = f"../{detect_faces_folder}/{movie}"
# des_emb_faces = f"../{faces_emb_folder}/{movie}"
# des_img_faces = f"../{faces_folder}/{movie}"
# extract_faces_emb(source_det_faces, des_emb_faces, des_img_faces , "VGG-Face")